#Imports

In [ ]:
from nltk.metrics.distance import edit_distance
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import itertools

pd.set_option('display.max_rows', None)

#Les bases: Distance Edit

In [ ]:
# #plus c'est bas, plus ils sont proches === distance
distance_1=edit_distance("venir", "venir")
distance_2=edit_distance("vunar", "venir")
print(f"Distance levenshtein entre 'venir' et 'venir' est {distance_1}\n"
      f"Distance levenshtein entre 'vunar' et 'venir' est  {distance_2}")

In [ ]:
#Essayez avec vos propres mots



In [ ]:
# # plus c'est haut, plus ils sont similaires === similarité
# distance_1_perc=edit_distance("venir", "venir")
# distance_2_perc=edit_distance("vunar", "venir")
print(f"Distance en pourcentage entre 'venir' et 'venir' est {1 - distance_1_perc / max(len('venir'), len('venir'))}\n"
      f"Distance en pourcentage entre 'vunar' et 'venir' est {1 - distance_2_perc / max(len('vunar'), len('venir'))}")

In [ ]:
#Essayez avec vos propres mots
#copiez le code dans la cellule précédent; remplacez les deux mots 'venir' et 'vunar' par vos mots



In [ ]:
#Essayons plusieurs mots pour voir si nous pouvons obtenir un score de proximté

texte1 = "Je suis convaincu que Dakar reste l'une des plus belles villes d'Afrique"
texte2 = "Dakar reste les villes les plus attirantes dans le monde"
score = 0
mots_differents={}
mots_proches={}

# Compter le nombre total de comparaisons à effectuer
totalité_de_comparaison = len(texte1.split()) * len(texte2.split())

for mot1 in texte1.split():
    for mot2 in texte2.split():
        similarité = 1 - edit_distance(mot1, mot2) / max(len(mot1), len(mot2))
        score += similarité
        if similarité < 0.7:
          mots_differents[mot1]=[mot2]
        else:
          mots_proches[mot1]=[mot2]

# Normaliser le score afin d'avoir une similarité entre 0 et 1
score_normalisé = 1- (score / totalité_de_comparaison)
print(f" Les deux textes sont similaires à {round(score_normalisé*100, 2)}%")

In [ ]:
mots_differents #voyons les mots les plus proches

In [ ]:
mots_proches #voyons les mots les plus distincts

#Vérifions si deux mots sont des cognats

In [ ]:
def vérifie_cognacité(mot1,mot2,edit_distance=edit_distance, seuil = 0.7):
    # Calculater la distance entre deux mots
    distance = edit_distance(mot1.lower(), mot2.lower())
    longeur_max = max(len(mot1), len(mot2))

    # Calculer la simialrité
    similarité = 1 - (distance / longeur_max)

    # Definissez un seuil de cognacité

    if similarité >= seuil:
        return f"'{mot1}' et '{mot2}' sont probablement des cognats (similarité: {similarité:.2f})."
    else:
        return f"'{mot1}'et '{mot2}' semblent ne pas être des cognats (similarité: {similarité:.2f})."

In [ ]:
vérifie_cognacité("aller", "alleri")

In [ ]:
#Essayez avec vos propres mots
#copiez le code dans la cellule précédent; remplacez les deux mots 'venir' et 'vunar' par vos mots






#Identification de cognates

In [ ]:
#chargez les données
!wget --no-check-certificate 'https://drive.google.com/uc?export=download&id=1Rp2NRHc3ZbwG-sEU5kt4126HSVPqyjnZ' -O cognate_data_1.csv

In [ ]:
#chargons les données
df_1=pd.read_csv("cognate_data_1.csv").drop("Unnamed: 0", axis=1)
df_1.head()

In [ ]:
#regardons les données de façon randomisées
indices_randomisés = random.sample(range(len(df_1)), 2)
lignes_randomisés = df_1.iloc[indices_randomisés]
print(lignes_randomisés)

Comment calculer la distance Edit entre plusieurs mots?
* prenez chaque mot et son sens
* si les deux mots on la même signification
* calculer la similarité entre les deux mots
* sauvegarder le résultats dans une matrice

In [ ]:
#Fonction pour calculer distance entre les mots ayant le même sens
def calculer_matrice(df, edit_distance=edit_distance):
    langues = df["Language"].unique()   #récupérer les langues dans vos données
    sens = df["Meaning"].unique()       #récupérer les significations disponible dans vos données
    mots = df["Word"].values            #récupérer les mots disponible dans vos données

    # Initialiser une matrice de distance avec un  NaN pour les cases vides, qui vous permet de sauvegarder les scores de similarité
    nombre_mots = len(mots)
    matrice_de_distance = np.full((nombre_mots, nombre_mots), np.nan)

    # Dictionaire pour lier chaque mot à son sens
    mot_vers_sens = df.set_index("Word")["Meaning"].to_dict()

    # Calculer distance entre les mots ayant même sens
    for i in range(nombre_mots):
        matrice_de_distance[i][i] = 1.0
        # le score des diagonals
        for j in range(i + 1, nombre_mots):
            if mot_vers_sens [mots[i]] == mot_vers_sens [mots[j]]:  # vérification de l'égalité des sens
                print(f"En train de calculer distance entre '{mots[i]}' et '{mots[j]}' (Sens: {mot_vers_sens [mots[i]]})")

                #calculer distance, puis similarité
                distance = edit_distance(mots[i], mots[j])
                longeur_max = max(len(mots[i]), len(mots[j]))
                similarité = 1 - (distance / longeur_max)

                #sauvegarder les résultas
                matrice_de_distance[i][j] = similarité
                matrice_de_distance[j][i] = similarité

    return pd.DataFrame(matrice_de_distance, columns=mots, index=mots)

In [ ]:
matrice_de_distance = calculer_matrice(df_1) #appeler la fonction afin d'avoir le calcul des distances

In [ ]:
matrice_de_distance.head(5)  #regarder ce à quoi ressemble les distances calculer

Comment voir quels mots ont été identifiés comme étant des cognats possibles?
* prenez les données de départ
* prenez la matrice detenant les scores de similarité
* filtrez les scores selon un seuil que vous préferez
* en croisant les deux, pour chaque mot, ayant une signification, dans une langue donnée, récupérez le score avec d'autres mots ayant le même sens dans d'autres langues afin de constituer un nouveau ensemble de données

In [ ]:
#filtrer les données selon un seuil pour obtenir des cognats
def filtrer_distances(df, distance_matrix, seuil=0.7):
    données_filtrées = []
    nombre_de_mots = len(df)

    for i in range(nombre_de_mots):
        for j in range(nombre_de_mots):
            if i != j and distance_matrix.iloc[i, j] >= seuil:
                données_filtrées.append({
                    "Meaning": df.iloc[i]["Meaning"],
                    "Word 1": df.iloc[i]["Word"],
                    "Language 1": df.iloc[i]["Language"],
                    "Word 2": df.iloc[j]["Word"],
                    "Language 2": df.iloc[j]["Language"],
                    "Edit Distance": distance_matrix.iloc[i, j]
                })

    return pd.DataFrame(données_filtrées)

In [ ]:
# Créer un dataframe filtré
données_filtrées = filtrer_distances(df_1, matrice_de_distance)
print(f"Les données de départ sont au total {len(df_1)}\nLes cognats sont au nombre total de {len(données_filtrées)}")

Comment analyser les résultats obtenus?
* vous pouvez regardez de façon randomisée les cognats
* vous pouvez récupérer la liste de cognats, l'imprimer, pour ensuite l'inspecter
* vous pouvez visualiser des statistiques concernant les résultats obtenus
* vous pouvez analyser les mots ayant été detectés comme étant des non-cognats

In [ ]:
#1. visualisation randomisée des résultats

In [ ]:
données_filtrées[données_filtrées["Meaning"]==random.choice(données_filtrées["Meaning"].unique())]

In [ ]:
#2. Imprimez les résultats pour inspecter les cognats detectés

In [ ]:
#obtenez un dictionnaire de cognats
def cognates_par_sens(filtered_df):

    cognates_dict = {}

    # Itérer à travers les données
    for _, ligne in filtered_df.iterrows():
        sens = ligne["Meaning"]
        cognat_pair = (ligne["Word 1"], ligne["Word 2"], ligne["Language 1"], ligne["Language 2"], ligne["Edit Distance"])

        if sens not in cognates_dict:
            cognates_dict[sens] = []

        cognates_dict[sens].append(cognat_pair)

    return cognates_dict

In [ ]:
cognats_selon_sens = cognates_par_sens(données_filtrées)

# Obtenez liste des cognats
for signe, cognats in cognats_selon_sens.items():
    print(f"Sens: {signe}")
    for cogn in cognats:
        print(f"  Mot 1: {cogn[0]}, Langue 1: {cogn[2]}, Mot 2: {cogn[1]}, Langue 2: {cogn[3]}, Distance: {cogn[4]}")

In [ ]:
#3. visualisez des statistiques concernant les cognats detectés

In [ ]:
#Comptez le nombre de cognats par sens
nombre_de_cognats = données_filtrées["Meaning"].value_counts()

# tracer une illustration
plt.figure(figsize=(12, 6))
sns.barplot(x=nombre_de_cognats.index, y=nombre_de_cognats.values/2)
plt.title("Nombre de cognats par sens")
plt.xlabel("Sens")
plt.ylabel("Nombre de cognats")
plt.xticks(rotation=45)
plt.show()

In [ ]:
signification = list(cognats_selon_sens.keys())
nombre_de_cognats_2 = [len(cognats_selon_sens[sens]) for sens in signification]

# Identification des mots les plus fréquents selon signification
mot_le_plus_fréquent = []
frequences = []

for sens, cogns in cognats_selon_sens.items():
    # récupérer tout les mots pour ce sens
    mots = [pair[0] for pair in cogns] + [pair[1] for pair in cogns]

    # Identifier le mot le plus fréquent et sa signification
    comptage_mots = Counter(mots)
    mot_freq, freq = comptage_mots.most_common(1)[0]

    mot_le_plus_fréquent.append(mot_freq)
    frequences.append(freq)

# tracer le graphique pour le nombre de cognat par signification
plt.figure(figsize=(12, 6))
plt.scatter(signification, nombre_de_cognats_2, color='blue', label="Nombre de cognats par signification")
plt.xlabel("Signification")
plt.ylabel("Nombre de Cognats")
plt.title("Nombre de Cognats par Signification")
plt.grid(True)

plt.xticks(rotation=90)

# tracer le graphique pour le mot le plus frequent par signification
plt.scatter(signification, frequences, color='red', label="Fréquence du mot le plus disponible")
for i, mot in enumerate(mot_le_plus_fréquent):
    plt.annotate(mot, (signification[i], frequences[i]), textcoords="offset points", xytext=(5,5), ha='center', fontsize=9)

plt.legend()
plt.show()

In [ ]:
# 4. analyser les non-cognats

In [ ]:
sens_non_cognats=[signification for signification in df_1["Meaning"].unique() if signification not in données_filtrées["Meaning"].unique()]
df_1[df_1["Meaning"].isin(sens_non_cognats)]

### Exercice:
* Mettez vous sur Google sheets (https://workspace.google.com/intl/fr/products/sheets/)
* Préparez une liste de mots dans deux ou trois langues en respectant le format suivant  ID|Langue|Signification|Mot
* Pour avoir des meilleurs résultats, vous pouvez mettre la forme API des mots dans la colonne 'Mot' - vous pouvez aussi utiliser la forme orthographique, puis vous appuyer sur les codes ci-dessous pour convertir et pondérer des formes en API avec de procéder avec la détection des cognats.
* Chargez vos données à la place des données utilisées pour la démonstration
* Remplacez 'Language' partout dans le code, avec 'Langue'; 'Meaning' avec 'Signification' et 'Word' avec 'Mot'
* Essayez d'exécuter les codes afin de detecter les cognats potentiels dans vos données

#Amélioration de l'algorithme en utilisant des formes en Alphabet Phonétique International

In [ ]:
# créer un tableau des sons avec lieu d'articulation - substituer des formes orthographiques si besoin
lieu = {
    "bilabial": ["b", "p", "β", "ɸ", "m", "", ""],
    "alvéolaire": ["d", "t", "z", "s", "n", "l", ""],
    "alvéolaire_2":["", "", "", "","", "", "r"],
    "palato-alvéolaire": ["", "", "ʒ", "ʃ", "", "", ""],
    "palatal": ["ɟ", "c", "y", "ç", "ɲ", "",""],
    "vélar": ["ɡ", "k", "ɣ", "x", "ŋ", "",""],
    "glottal": ["", "", "ɦ", "h", "", "",""]
}

# Definir les modes d'articulation
mode = ['occlusive voisée', 'occlusive non-voisée', 'fricative voisée', 'fricative non-voisée', 'nasale', 'latéral', 'latéral']

# créer la carte des sons
carte_de_sons= pd.DataFrame(lieu, index=mode)
carte_de_sons

In [ ]:
# calculer la distance entre des sons
def calculer_distance(col1, col2, idx1, idx2, son1, son2):

    if son1 == son2:  #  sons identiques, retourner 0
        return 0
    elif col1 == col2:#  sons ayant le même lieux d'articulation, retourner une distance de 0.3
        return 0.5
    elif idx1 == idx2:# sons ayant différents lieux d'articulation, mais même mode d'articulation, retourner une distance progressive
        col_distance = abs(col1 - col2) # Calculer la distance entre deux colonnes et augmente selon éloignment
        return min(1, 0.5 + (0.1 * col_distance))
    else:              # sont ayant différent lieux d'articulation et mode d'articulation
        return 1

In [ ]:
#préparer les pairs des sons
sons_paires = {}
colonnes = list(carte_de_sons.columns)
for (idx1, ligne1), (idx2, ligne2) in itertools.product(carte_de_sons.iterrows(), repeat=2):
    for col1, son1 in ligne1.items():
        for col2, son2 in ligne2.items():
            if son1 and son2:  # ne pas prendre en considération des cellulles vides
                paire = tuple(sorted((son1, son2))) # classez les paires pour s'assurer de la non-duplicité des clés dans le dico
                if paire not in sons_paires:
                    # Calculer les positions des colonnes et lignes pour établir la distance
                    col_idx1, col_idx2 = colonnes.index(col1), colonnes.index(col2)
                    sons_paires[paire] = calculer_distance(col_idx1, col_idx2, idx1, idx2, son1, son2)

In [ ]:
def levenshtein_avec_coût(mot1, mot2, sons_paires=sons_paires, seuil=0.7):
    m, n = len(mot1), len(mot2)
    dp = [[0 for _ in range(n + 1)] for _ in range(m + 1)]

    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0:
                dp[i][j] = j
            elif j == 0:
                dp[i][j] = i
            else:
                # Initializer un coût de substitution
                coût_de_similarité = 1  # coût par défaut si probabilité à priori n'existe pas dans sons_paires

                # même sons, pas de coût
                if mot1[i - 1] == mot2[j - 1]:
                    coût_de_similarité = 0
                else:
                    # différents sons; récupérer probabilités de sons_paires
                    coût_de_similarité = sons_paires.get((mot1[i - 1], mot2[j - 1]),
                                                      sons_paires.get((mot2[j - 1], mot1[i - 1]), 1))

                dp[i][j] = min(dp[i - 1][j] + 1,  # suppression
                               dp[i][j - 1] + 1,  # insertion
                               dp[i - 1][j - 1] + ((coût_de_similarité)/seuil))  # substitution
    distance_temp=dp[m][n] #peut dépasser 2

    return distance_temp

In [ ]:
print("Résultats sans intégration des connaissances de la linguistique typologique")
print("*" * 70)

word_pairs = [("addel", "atler"), ("addel", "adhel"),
              ("addel", "athel"), ("addel", "attel"),
              ("addel", "addel")]

for mot1, mot2 in word_pairs:
    print(vérifie_cognacité(mot1, mot2))

In [ ]:
print("Résultats avec intégration des connaissances de la linguistique typologique")
print("*" * 70)
for mot1, mot2 in word_pairs:
    print(vérifie_cognacité(mot1, mot2, edit_distance=levenshtein_avec_coût))